In [85]:
import random
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
import scipy
from scipy.integrate import quad

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [11]:
"""Calculate the next statement

An implementation of standard runge_kutta methods.

Args:
    function: 
    x_n: The current state variable.
    t_n: The current timestamp.
    h: time step size
    
Returns:
    x_next: The next state variable    

"""
def runge_kutta(x_n, t_n, h, alpha):
    def function(x, t, a):
        return -a * x * np.exp(-a * t)
    k_1 = function(x_n, t_n, alpha)
    k_2 = function(x_n + k_1 * h / 2, t_n + h/2, alpha)
    k_3 = function(x_n + k_2 * h / 2, t_n + h/2, alpha)
    k_4 = function(x_n + k_3 * h, t_n + h, alpha)
    x_next = x_n + h * (k_1 + 2 * k_2 + 2 * k_3 + k_4) / 6
    return x_next

In [12]:
"""
G FCN : DEFINE FORWARD MODEL 
"""
def ode_model(t,x,alpha):
    dxdt = np.zeros((1,len(x)))
    dxdt[0,0] =  -alpha*x[0] 
    return [np.transpose(dxdt)];


In [67]:
"""
G FCN: DEFINE X DATA GENERATION 
"""
def state_predictor(x0,alpha,dj): 
    tspan =  [0,dj];
    t_eval = np.array([np.linspace(0,dj,10)]) #return response at 10 points from t=0 to t=tfinal
    x0 = [x0];
    alpha = alpha
    ode_sol = scipy.integrate.solve_ivp(lambda t,x: ode_model(t,x,alpha), tspan, x0, method='LSODA') 
    #LSODA is the most consistent solver
    times = ode_sol.t; xs = ode_sol.y
    #plt.figure
    #plt.plot(times,xs.transpose()[:,0])
    return xs.transpose()[-1,0]


In [68]:
### state_predictor(0.5 ,1.0, 0.1)

In [69]:
def state_predictor(x0,alpha,dj): 
        tspan =  [0,dj];
        t_eval = np.array([np.linspace(0,dj,10)]) #return response at 10 points from t=0 to t=tfinal
        x0 = [x0];
        alpha = alpha
        ode_sol = scipy.integrate.solve_ivp(lambda t,x: ode_model(t,x,alpha), tspan, x0, method='LSODA') 
        #LSODA is the most consistent solver
        times = ode_sol.t; xs = ode_sol.y
        return xs.transpose()[-1,0]


In [8]:
"""The equations from Example 1.
Implement the equations from Example 1, which 

Args:
    alpha: A random coefficient.
    t: Time.
    x: The state variable. 

Returns:
    func(x_0, alpha, t): The function to calculate the state.
"""
def linear_scalar_equation(x, alpha, t):
    return -alpha * x * np.exp(-alpha * t)

In [4]:
"""
Generate training data with standard runge kutta methods.

Args:
    n: The number of the generated data, 20 times of the number of parameters.
    x_0: The initial state
    step_size: 
    
Returns:
    data: A list of generated data. 
"""
def data_generating(n, x_0, step_size):
    dataset = []
    for i in range(20 * n):
        
        alpha = random.random() # A random number between 0 to 1
        time = random.randrange(300)
        x = linear_scalar_equation(alpha, time, x_0)
        delta = 300
        x_next = x
        for j in range(delta):
            x_next = runge_kutta(linear_scalar_equation(x_next, alpha, time), x_next, time, step_size)
            time += 1
        dataset.append([[x, alpha, delta], x_next])
    return dataset

In [70]:
def Train_Data_Gen(Nsamples):
    """
    G FCN: GENERATE DATA
    """
    Nsamples = 100;
    dataset = []
    P_lb = 0.0;
    P_ub = 1.0;
    
    S_lb = 0.1;
    S_ub = 0.8;
    
    for i in range(Nsamples):
        alpha = np.random.uniform(P_lb,P_ub)
        delta = 0.1 #in principle random 
        x0 =  np.random.uniform(S_lb,S_ub)
        x = state_predictor(x0, alpha, delta)
        x_next = data_generating(x0, alpha, delta)
        dataset.append([[x0, alpha, delta], x_next])
    return dataset

In [75]:
TrainData = np.array(dataset)
Xdata = TrainData[:,0];
Ydata = TrainData[:,1]

In [90]:
#alpha = 1
#data_training = data_generating(n=300, x_0=1, step_size=0.1)
#train_loader = data_generating(20, 1, 0.1)

## GENERATES TRAINING DATA###
N_Train = 1000
train_loader = Train_Data_Gen(N_Train)
learning_rate = 0.2

In [73]:
"""A Neural Net with three FC layers with Tanh functions betwween them as activation functions.

"""
class Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.Tanh())
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))

    def forward(self, x):
        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        return x3 + x #Only add the first feature, x0, not all of them 


In [87]:
model = Net(3, 40, 40, 1)
if torch.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss() #Maybe MSELoss: suitable for regression 
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

epoch = 0
for data in train_loader:
    x,y = data
    x = np.array(x)
    x = Variable(torch.from_numpy(x)) #Fix this part, x is initially 
    
    # x_j, alpha_j, edlta_j = x
    out = model(x)
    loss = criterion(out, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1    #Maybe add batch size 
    if epoch%50 == 0:
        print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #2 'mat2' in call to _th_mm

# TO DO LIST

1. Fix the features issue for the Reset (should only read the first feature, x0) 
2. x = Variable(torch.from_numpy(x)) #Fix this part, x is initially a list, how to properly pass it?
3. Check batch size
4. Check MESLoss (regression) (I think crossentropy is for class/ion) 
5. When the net bugs are fixed, train and get validation & test error plots (w/ n of epochs)
6. Add a dropout layer (prob ~0.3) and see what happens with errors 
7. Check how we can add a custom activation fcn 



In [91]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})


NameError: name 'get_custom_objects' is not defined